<a href="https://colab.research.google.com/github/chldydgh4687/PRproject/blob/5.0_release/Bowspatialpyramidmatching(0.5_not%20spmkernel).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

-rw-r--r-- 1 root root 66 Dec  4 05:07 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=3f2712ededeb1040c54e4a39fdbbb4b683b1a9a64724c2815f350627cb9139f9
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
Kaggle API 1.5.6


In [0]:
! kaggle competitions download -c 2019-ml-finalproject

 86% 56.0M/65.0M [00:00<00:00, 98.1MB/s]
100% 65.0M/65.0M [00:00<00:00, 141MB/s] 


In [0]:
import zipfile
import os

os.mkdir('/content/input')

zip_ref = zipfile.ZipFile("/content/2019-ml-finalproject.zip", 'r')
zip_ref.extractall("/content/input")
zip_ref.close()

In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-3.4.7.28:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-3.4.7.28.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtCore-9549151f.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtGui-6d0f14dd.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavcodec-eac15e48.so.58.21.104
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavformat-b6bcbe33.so.58.17.101
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavutil-e1b1a17d.so.56.18.102
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswresample-b4363bfa.so.3.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswscale-15b3fdc6.so.5.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libvpx-81a43c0a.so.5.0.0
Proceed (y/n)?   Successfully uninstalled opencv-python-3.4.7.28
Uninstalling opencv-contrib-python-3.4.3.

In [0]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import torch
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook as tqdm

import glob

In [0]:
df_data = pd.read_csv('/content/input/Label2Names.csv', header=None)

In [0]:
cv2.__version__

'3.4.2'

In [0]:
def TrainDenseSIFT(path):
  data_des = list()
  data_images = list()
  data_label = list()
  data_set = list()

  Step_size = 16
  img_size = 256
 
  for cls in tqdm(os.listdir(path)):
    img_list = os.listdir(path+'/'+cls)
    img_list.sort()
    if cls == 'BACKGROUND_Google':
      label = 102
    else :
      label = (df_data.index[df_data[1]==cls] +1).tolist()[0]


    for img in img_list:

      image = cv2.imread(path + '/' + cls + '/' + img)
     
      gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
      gray = cv2.resize(gray, (img_size, img_size))
      sift = cv2.xfeatures2d.SIFT_create()
#compute dense
      kp = [cv2.KeyPoint(x,y, Step_size) for y in range(0, gray.shape[0], Step_size) for x in range(0, gray.shape[1],Step_size)]
      keypoint, des = sift.compute(gray, kp)
      
      data_set.append(des)
      data_des.append(des)
      data_images.append(gray)
      data_label.append(label)

  return data_set, data_des, data_images, data_label

In [0]:
def TestDenseSIFT(path):

  img_size = 256
  Step_size = 16
  test_des = list()
  test_images = list()
  test_set = list()

  img_list = os.listdir(path)
  img_list.sort()

  for img in tqdm(img_list):
    image = cv2.imread(path + '/' + img)
    
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (img_size, img_size))
  
    sift = cv2.xfeatures2d.SIFT_create()

    kp = [cv2.KeyPoint(x,y, Step_size) for y in range(0, gray.shape[0], Step_size) for x in range(0, gray.shape[1],Step_size)]
    keypoint, des = sift.compute(gray, kp)

    test_set.append(des)
    test_des.append(des)
    test_images.append(gray)

  return test_set, test_des, test_images

In [0]:
train_data, train_des, train_image, train_label = TrainDenseSIFT("/content/input/train")

In [0]:
test_data, test_des, test_image = TestDenseSIFT("/content/input/testAll_v2")

KeyboardInterrupt: ignored

In [0]:
train_des = np.array(train_des)
train_des.shape

(3060, 256, 128)

In [0]:
test_des = np.array(test_des)
test_des.shape

(1692, 256, 128)

In [0]:
train_des=train_des.reshape(-1,128)
train_des.shape

(783360, 128)

In [0]:
test_des = test_des.reshape(-1,128)
test_des.shape

(433152, 128)

In [0]:
import math

def extract_denseSIFT(img):
  Step_size = 8
  sift = cv2.xfeatures2d.SIFT_create()
#compute dense
  kp = [cv2.KeyPoint(x,y, Step_size) for y in range(0, img.shape[0], Step_size) for x in range(0, img.shape[1],Step_size)]
  keypoint, des = sift.compute(img, kp)

  return des


# form histogram with Spatial Pyramid Matching upto level L with codebook kmeans and k codewords
def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[0]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                desc = extract_denseSIFT(img[y:y+h_step, x:x+w_step])                
                #print("type:",desc is None, "x:",x,"y:",y, "desc_size:",desc is None)
                predict = kmeans.predict(desc)
                histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
                weight = 2**(l-L)
                h.append(weight*histo)
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    # normalize hist
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist


# get histogram representation for training/testing data
def getHistogramSPM(L, data, kmeans, k):    
    x = []
    for i in range (len(data)):        
        hist = getImageFeaturesSPM(L, data[i], kmeans, k)        
        x.append(hist)
    return np.array(x)

In [0]:
pip install kmc2

     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 163kB 9.0MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252240 sha256=e86c3f8f65cfa274877c2934a6246864230531c76cb9aa73455c3e5594885521
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
import kmc2 
from sklearn.cluster import MiniBatchKMeans
import copy

In [0]:
seeding = kmc2.kmc2(train_des, 200)
kmeans = MiniBatchKMeans(200,init=seeding).fit(train_des)
codebook = kmeans.cluster_centers_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  


In [0]:
train_image = np.array(train_image)
train_image.shape

(3060, 256, 256)

In [0]:
train_histo = getHistogramSPM(2, train_image, kmeans, 200)


KeyboardInterrupt: ignored

In [0]:
test_histo = getHistogramSPM(2, test_image, kmeans, 200)

In [0]:
train_scaler = StandardScaler().fit(train_histo)
train_histo = train_scaler.transform(train_histo)

test_histo = train_scaler.transform(test_histo)

In [0]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA

from sklearn.pipeline import make_pipeline

pca = RandomizedPCA(n_components=200, whiten=True, random_state=42)
svc = SVC(kernel='precomputed', class_weight='balanced')
model = make_pipeline(pca, svc)

In [0]:
from sklearn.model_selection import GridSearchCV


param_grid = {'svc__C': [0.01, 0.05, 0.1, 0.5 , 1],
              'svc__gamma': [0.00001, 0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(train_histo, train_label)
print(grid.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: ignored

In [0]:
model = grid.best_estimator_
yfit = model.predict(test_histo)
yfit.shape

(1692,)

In [0]:
img_list = os.listdir("/content/input/testAll_v2")
img_list.sort()

In [0]:
yfit = yfit.reshape(-1,1)
result_img_list = np.array(img_list).reshape(-1,1)
total_result = np.hstack([result_img_list,yfit])

In [0]:
df = pd.DataFrame(total_result, columns=["Id","Category"])
df.to_csv('results-yongho-v3.csv',index=False, header=True)

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-yongho-v3.csv -m "Final_Term_Project"

100% 29.6k/29.6k [00:00<00:00, 134kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 

## #0 (Developing)

### #0 Using Google Colab with GitHub ( Git Clone - branch : Develop )

In [0]:
!git config --global user.email "s2blue4687@naver.com"
!git config --global user.name "chldydgh4687"
!git clone -b Develop https://github.com/chldydgh4687/PRproject.git

### #0 code upload (Release - branch : Develop)

![대체 텍스트](https://github.com/chldydgh4687/PRproject/blob/Develop/study/picture/23423423.PNG?raw=true)

### #0 code upload (branch merge / push, pull - Gitkraken )

## #0 (Release) Using Google Colab with GitHub ( Git Clone - branch : Master )

In [0]:
!git config --global user.email "s2blue4687@naver.com"
!git config --global user.name "chldydgh4687"
!git clone -b master https://github.com/chldydgh4687/PRproject.git

# ##import Library

In [0]:
from scipy import misc
lena = misc.face()
kernel = np. outer(signal.gaussian(70,8), signal.gaussian(70,8))
blurred = signal.fftconvolve(lena, kernel, mode='same')

ValueError: ignored

In [0]:
from scipy import signal
from scipy import misc
lena = misc.face()
scharr = np.array([[ -3-3j, 0-10j,  +3 -3j],
                   [-10+0j, 0+ 0j, +10 +0j],
                   [ -3+3j, 0+10j,  +3 +3j]]) # Gx + j*Gy
grad = signal.convolve2d(np.array(lena), scharr, boundary='symm', mode='same')

ValueError: ignored